In [8]:
import numpy as np

def generate_points(num_samples=100):  # 기본 샘플 수를 인자로 받음
    # Collocation points
    x_collocation = np.random.uniform(-1, 3, size=(num_samples * 5, 1))  # 500 -> num_samples * 5
    y_collocation = np.random.uniform(-1, 1, size=(num_samples * 5, 1))
    t_collocation = np.random.uniform(0, 10, size=(num_samples * 5, 1))
    collocation_points = np.concatenate([x_collocation, y_collocation, t_collocation], axis=1)

    # Boundary and initial points, 각각 num_samples로 조정
    y_inlet = np.random.uniform(-1, 1, size=(num_samples, 1))
    t_inlet = np.random.uniform(0, 10, size=(num_samples, 1))
    inlet_points = np.concatenate([-np.ones((num_samples, 1)), y_inlet, t_inlet], axis=1)

    y_outlet = np.random.uniform(-1, 1, size=(num_samples, 1))
    t_outlet = np.random.uniform(0, 10, size=(num_samples, 1))
    outlet_points = np.concatenate([3*np.ones((num_samples, 1)), y_outlet, t_outlet], axis=1)

    x_wall_top = np.random.uniform(-1, 3, size=(num_samples, 1))
    t_wall_top = np.random.uniform(0, 10, size=(num_samples, 1))
    wall_top_points = np.concatenate([x_wall_top, np.ones((num_samples, 1)), t_wall_top], axis=1)

    x_wall_bottom = np.random.uniform(-1, 3, size=(num_samples, 1))
    t_wall_bottom = np.random.uniform(0, 10, size=(num_samples, 1))
    wall_bottom_points = np.concatenate([x_wall_bottom, -np.ones((num_samples, 1)), t_wall_bottom], axis=1)

    x_initial = np.random.uniform(-1, 3, size=(num_samples, 1))
    y_initial = np.random.uniform(-1, 1, size=(num_samples, 1))
    initial_points = np.concatenate([x_initial, y_initial, np.zeros((num_samples, 1))], axis=1)

    return collocation_points, inlet_points, outlet_points, wall_top_points, wall_bottom_points, initial_points

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Define the neural network for u, v, and p
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        # 중간 레이어
        self.middle_layers = nn.ModuleList()
        for i in range(len(layers)-2):
            self.middle_layers.append(nn.Linear(layers[i], layers[i+1]))
            self.middle_layers.append(nn.Tanh())
        
        # 마지막 레이어를 분리하여 u, v와 p를 각각 예측
        self.last_layer_uv = nn.Linear(layers[-2], 2) # u와 v를 위한 레이어
        self.last_layer_p = nn.Linear(layers[-2], 1) # p를 위한 레이어

    def forward(self, x):
        for layer in self.middle_layers:
            x = layer(x)
        u_v = self.last_layer_uv(x)
        p = self.last_layer_p(x)
        return u_v, p


def compute_pde_loss(model, inputs, nu):
    inputs.requires_grad_(True)
    u_v, p = model(inputs)
    u = u_v[:, 0].unsqueeze(1)
    v = u_v[:, 1].unsqueeze(1)
    
    # Compute gradients
    ones = torch.ones_like(u)
    u_x = torch.autograd.grad(u, inputs, grad_outputs=ones, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u, inputs, grad_outputs=ones, create_graph=True)[0][:, 1].unsqueeze(1)
    u_t = torch.autograd.grad(u, inputs, grad_outputs=ones, create_graph=True)[0][:, 2].unsqueeze(1)
    
    v_x = torch.autograd.grad(v, inputs, grad_outputs=ones, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v, inputs, grad_outputs=ones, create_graph=True)[0][:, 1].unsqueeze(1)
    v_t = torch.autograd.grad(v, inputs, grad_outputs=ones, create_graph=True)[0][:, 2].unsqueeze(1)
    
    p_x = torch.autograd.grad(p, inputs, grad_outputs=ones, create_graph=True)[0][:, 0].unsqueeze(1)
    p_y = torch.autograd.grad(p, inputs, grad_outputs=ones, create_graph=True)[0][:, 1].unsqueeze(1)
    
    # Navier-Stokes equations
    continuity_eq = u_x + v_y
    u_momentum_eq = u_t + u*u_x + v*u_y + p_x - nu*(torch.autograd.grad(u_x, inputs, grad_outputs=ones, create_graph=True)[0][:, 0].unsqueeze(1) + torch.autograd.grad(u_y, inputs, grad_outputs=ones, create_graph=True)[0][:, 1].unsqueeze(1))
    v_momentum_eq = v_t + u*v_x + v*v_y + p_y - nu*(torch.autograd.grad(v_x, inputs, grad_outputs=ones, create_graph=True)[0][:, 0].unsqueeze(1) + torch.autograd.grad(v_y, inputs, grad_outputs=ones, create_graph=True)[0][:, 1].unsqueeze(1))
    
    # PDE loss
    pde_loss = torch.mean(continuity_eq**2) + torch.mean(u_momentum_eq**2) + torch.mean(v_momentum_eq**2)
    return pde_loss

def compute_bc_loss(model, inlet_points, outlet_points, wall_top_points, wall_bottom_points):
    # Inlet condition: u = 1, v = 0
    inlet_u_v, _ = model(torch.tensor(inlet_points, dtype=torch.float32))
    inlet_loss = torch.mean((inlet_u_v[:, 0] - 1) ** 2) + torch.mean(inlet_u_v[:, 1] ** 2)

    # Outlet condition: p = 0 (Assuming model returns u, v, and p separately)
    _, outlet_p = model(torch.tensor(outlet_points, dtype=torch.float32))
    outlet_loss = torch.mean(outlet_p ** 2)

    # No-slip wall condition: u = v = 0
    wall_top_u_v, _ = model(torch.tensor(wall_top_points, dtype=torch.float32))
    wall_bottom_u_v, _ = model(torch.tensor(wall_bottom_points, dtype=torch.float32))
    wall_loss = torch.mean(wall_top_u_v ** 2) + torch.mean(wall_bottom_u_v ** 2)

    # Total BC loss
    bc_loss = inlet_loss + outlet_loss + wall_loss
    return bc_loss

def compute_ic_loss(model, initial_points):
    """
    Compute the initial condition loss.
    
    Parameters:
    - model: The PINN model.
    - initial_points: Tensor of input values at the initial time (x, y, t=0).
    
    Returns:
    - ic_loss: The computed initial condition loss.
    """
    # 모델을 통해 초기 포인트에 대한 예측을 수행합니다.
    predicted_initial_u_v, _ = model(torch.tensor(initial_points, dtype=torch.float32))
    
    # 초기 조건에 대한 loss를 계산합니다. 여기서는 inlet 조건에 따라 u = 1, v = 0을 사용합니다.
    # 실제 초기 조건이 다르다면 이 부분을 조정해야 합니다.
    ic_loss = torch.mean((predicted_initial_u_v[:, 0] - 1) ** 2) + torch.mean(predicted_initial_u_v[:, 1] ** 2)
    
    return ic_loss


# Example usage
layers = [3, 50, 50, 50, 2] # The input layer includes x, y, t, and the output layer includes u, v, and p
model = PINN(layers=layers)
nu = 0.01 # Viscosity, example value

# Define inputs (x, y, t)
inputs = torch.rand(100, 3, requires_grad=True) # Example inputs

# Compute PDE loss
pde_loss = compute_pde_loss(model, inputs, nu)
print(f"PDE Loss: {pde_loss.item()}")

PDE Loss: 0.0061466386541724205


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

In [12]:
collocation_points, inlet_points, outlet_points, wall_top_points, wall_bottom_points, initial_points = generate_points(10000)
print(collocation_points.shape, inlet_points.shape, outlet_points.shape, wall_top_points.shape, wall_bottom_points.shape, initial_points.shape)

(50000, 3) (10000, 3) (10000, 3) (10000, 3) (10000, 3) (10000, 3)


In [ ]:
BATCH_SIZE = 10000
collocation_points_tensor = torch.tensor(collocation_points, dtype=torch.float32)
inlet_points_tensor = torch.tensor(inlet_points, dtype=torch.float32)
outlet_points_tensor = torch.tensor(outlet_points, dtype=torch.float32)
wall_top_points_tensor = torch.tensor(wall_top_points, dtype=torch.float32)
wall_bottom_points_tensor = torch.tensor(wall_bottom_points, dtype=torch.float32)
initial_points_tensor = torch.tensor(initial_points, dtype=torch.float32)

collocation_loader = DataLoader(collocation_points_tensor, batch_size=BATCH_SIZE, shuffle=True)
inlet_loader = DataLoader(inlet_points_tensor, batch_size=BATCH_SIZE, shuffle=True)
outlet_loader = DataLoader(outlet_points_tensor, batch_size=BATCH_SIZE, shuffle=True)
wall_top_loader = DataLoader(wall_top_points_tensor, batch_size=BATCH_SIZE, shuffle=True)
wall_bottom_loader = DataLoader(wall_bottom_points_tensor, batch_size=BATCH_SIZE, shuffle=True)
initial_loader = DataLoader(initial_points_tensor, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# 가정: optimizer, model, total_epochs 등이 이미 정의되어 있음
total_epochs = 100

for epoch in range(total_epochs):
    optimizer.zero_grad()
    
    # PDE Loss 계산
    pde_loss = compute_pde_loss(model, collocation_points, nu)
    
    # BC Loss 계산
    bc_loss = compute_bc_loss(model, inlet_points, outlet_points, wall_top_points, wall_bottom_points)
    
    # IC Loss 계산
    ic_loss = compute_ic_loss(model, initial_points)
    
    # Total Loss
    total_loss = pde_loss + bc_loss + ic_loss
    
    # Backpropagation and optimization
    total_loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Total Loss: {total_loss.item()}, PDE Loss: {pde_loss.item()}, BC Loss: {bc_loss.item()}, IC Loss: {ic_loss.item()}")
